In [5]:
import os

os.environ["DASHSCOPE_API_KEY"] = "sk-9041beedef014f4cb57ff1ef1dc6cd33"

from langchain_community.llms import Tongyi
from langchain_core.messages import HumanMessage, SystemMessage
model = Tongyi()

In [6]:
from langchain_core.messages import HumanMessage

model.invoke([HumanMessage(content="你好，我是周向子健")])

'你好，周向子健！有什么我可以帮你的吗？'

In [8]:
model.invoke([HumanMessage(content="我的名字是什么呢")])

'您在问题中并没有提供名字相关信息，所以我无法得知您的名字。如果您想告诉我您的名字，可以在此提供更多信息。'

In [10]:
from langchain_core.messages import AIMessage

model.invoke(
    [
        HumanMessage(content="你好我是周向子健"),
        AIMessage(content="你好，周向子健。今天我该怎么帮助你"),
        HumanMessage(content="我叫什么?"),
    ]
)

'你叫周向子健。'

In [14]:
from langchain_core.chat_history import (
    BaseChatMessageHistory,
    InMemoryChatMessageHistory,
)
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]


with_message_history = RunnableWithMessageHistory(model, get_session_history)

In [15]:

config = {"configurable": {"session_id": "abc2"}}

response = with_message_history.invoke(
    [HumanMessage(content="你好，我的名字叫周向子健！")],
    config=config,
)
print(response)

Error in RootListenersTracer.on_llm_end callback: KeyError('message')


你好，周向子健！很高兴认识你。你可以叫我Qwen。我是一个来自阿里云的大规模语言模型。我可以帮助你回答问题、提供信息或进行各种主题的对话。有什么我可以帮你的吗？


In [16]:

response = with_message_history.invoke(
    [HumanMessage(content="我的名字叫什么?")],
    config=config,
)
print(response)

Error in RootListenersTracer.on_llm_end callback: KeyError('message')


你好！你刚刚提到的名字是“Human”。不过，如果你有其他名字或者想让我称呼你别的名字，可以告诉我哦！


In [18]:
from langchain_community.llms import Tongyi
from langchain_core.messages import HumanMessage
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

# 提示模板
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "你是一个有用的助手。尽你所能回答所有问题。",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

# 历史会话存储
store = {}

# 获取会话历史
def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

# 使用 Tongyi LLM
model = Tongyi()

# 构建链式调用
chain = prompt | model

# 历史消息
with_message_history = RunnableWithMessageHistory(chain, get_session_history)

# 对话1
config = {"configurable": {"session_id": "abc2"}}

# 将一个消息列表传递给 .invoke 方法
response1 = with_message_history.invoke({"messages": [HumanMessage(content="你好，我是周向子健")]}, config=config)
print(response1)


你好，周向子健！很高兴认识你。有什么我可以帮你的吗？无论是问题解答、建议还是其他任何事情，我都会尽力协助你。


In [19]:
config = {"configurable": {"session_id": "abc2"}}
response2 = with_message_history.invoke({"messages":[HumanMessage(content="我的姓名是什么?")]}, config=config)
print(response2)

config2 = {"configurable": {"session_id": "abc3"}}
response2 = with_message_history.invoke({"messages":[HumanMessage(content="我的姓名是什么?")]}, config=config2)
print(response2)

你的姓名是周向子健。如果有其他问题或需要进一步的帮助，请随时告诉我！
对不起，我无法知道你的姓名。你可以告诉我你的姓名，我会记住它以便在我们的对话中使用。


In [21]:
from langchain_community.llms import Tongyi
from langchain_core.messages import HumanMessage,AIMessage
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnablePassthrough
# 获取会话历史
def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
            store[session_id] = ChatMessageHistory()
    return store[session_id]
# 历史消息
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)
# k=10 则无法记得姓名是什么，k=20 则可以记得
def filter_messages(messages, k=10):
    return messages[-k:]
chain = (
    RunnablePassthrough.assign(messages=lambda x: filter_messages(x["messages"]))
    | prompt
    | model
)
messages = [
    SystemMessage(content="你是一个非常棒的助手！"),
    HumanMessage(content="你好，我的名字叫小明！"),
    AIMessage(content="你好！"),
    HumanMessage(content="我喜欢香草冰淇淋！"),
    AIMessage(content="很好"),
    HumanMessage(content="2 + 2等于几？"),
    AIMessage(content="4"),
    HumanMessage(content="谢谢"),
    AIMessage(content="不客气"),
    HumanMessage(content="你开心吗"),
    AIMessage(content="当然"),
]
# 模型有历史聊天记录，再次提问
response = with_message_history.invoke(
    {"messages": messages + [HumanMessage(content="我的姓名是什么?")]}, 
    config=config
)
print(response)

你没有告诉我你的名字，所以我无法知道你的姓名。如果你愿意分享，我可以记住它，并在我们的对话中使用。


In [22]:
config = {"configurable": {"session_id": "abc2"}}
# 模型有历史聊天记录，再次提问
for r in  with_message_history.stream(
    {"messages":   [HumanMessage(content="请讲一个超过5句话的笑话给我听")]}, 
    config=config
):
    print(r,  width=200)

Error in RootListenersTracer.on_chain_end callback: ValueError('Expected str, BaseMessage, List[BaseMessage], or Tuple[BaseMessage]. Got None.')


TypeError: 'width' is an invalid keyword argument for print()

In [23]:
from langchain_community.llms import Tongyi
from langchain_core.messages import HumanMessage,AIMessage
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnablePassthrough

# 提示模板
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "你是一个有用的助手。尽你所能回答所有问题。",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

# 历史会话存储
store = {}

# 获取会话历史
def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

# 使用 Tongyi LLM
model = Tongyi()

# k=10 则无法记得姓名是什么，k=20 则可以记得
def filter_messages(messages, k=10):
    return messages[-k:]

chain = (
    RunnablePassthrough.assign(messages=lambda x: filter_messages(x["messages"]))
    | prompt
    | model
)

# 历史消息
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)

config = {"configurable": {"session_id": "abc2"}}

response = with_message_history.invoke(
    {"messages": [HumanMessage(content="我叫周文皓")]},
    config=config
)

print(response)

很高兴认识你，周文皓。有什么我可以帮你的吗？


In [25]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="我想学习粤语可以吗")]},
    config=config
)
print(response)

当然可以！学习粤语是一个非常有意义的选择，尤其是在与广东、香港和澳门等地的人交流时非常有用。我已经为你准备了一些学习粤语的建议：

### 1. **基础发音**
   粤语有九声六调，发音比普通话复杂一些。建议你先从基础的发音学起，尤其是声调。你可以通过听音频或视频来模仿发音。

   - **声母**：粤语有20多个声母，类似于普通话。
   - **韵母**：粤语的韵母比普通话多，尤其是鼻音和塞音收尾的韵母（如“-m”、“-p”、“-t”）。
   - **声调**：粤语有九个声调，分为六个主要声调和三个入声调。掌握声调是学习粤语的关键。

### 2. **常用词汇和短语**
   你可以从日常生活中常用的词汇和短语开始学习。例如：
   - 你好：你好呀（nei5 hou6 aa3）
   - 谢谢：多谢（do1 ze6）
   - 对不起：对不起（deoi6 baai2）
   - 请问：请问下（ching3 boon2 faat3）

### 3. **使用学习工具**
   - **手机应用**：有许多粤语学习的应用程序，比如“粤语课堂”、“HelloTalk”等，可以帮助你练习对话和听力。
   - **YouTube频道**：有很多粤语教学频道，提供免费的视频教程，适合初学者。
   - **字典和词典**：使用粤语字典（如“粤语在线词典”）可以帮助你查找单词的发音和意思。

### 4. **多听多说**
   - **看粤语电影或电视剧**：通过观看粤语电影、电视剧或综艺节目，你可以熟悉粤语的表达方式和常用词汇。
   - **找语言伙伴**：如果你有机会接触讲粤语的人，尽量多和他们交流，实践是最好的学习方法。

### 5. **参加课程**
   如果你有条件，可以报名参加线下的粤语课程，或者寻找在线的粤语老师进行一对一辅导。

### 6. **保持耐心**
   粤语的学习过程可能会有些挑战，尤其是声调和发音部分，但只要你坚持不懈，逐渐积累词汇和语感，一定会取得进步。

如果你有任何具体问题或需要进一步的帮助，随时告诉我！祝你学习顺利！😊

另外，你也可以告诉我你目前的汉语水平，这样我可以根据你的具体情况给出更有针对性的建议。


In [26]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="请问我是谁啊")]},
    config=config
)
print(response)

你之前提到你叫周文皓。如果你有其他问题或需要更多帮助，随时告诉我哦！😊
